In [5]:
import sys
sys.path.insert(0, '../')
import csv
import copy
import numpy as np
import aux 
from aux import *




import argparse,pwd,os,numpy as np,h5py
from os.path import splitext,exists,dirname,join,basename
from os import makedirs
import matplotlib.pyplot as plt
import h5py
from os.path import join,dirname,basename,exists,realpath
from os import makedirs
from tensorflow.examples.tutorials.mnist import input_data
from sklearn.metrics import roc_auc_score
import time
import os 
from tqdm import tqdm
import sys,seaborn as sns
import sklearn

W1225 21:36:00.404302 47717600894592 deprecation_wrapper.py:119] From ../aux.py:29: The name tf.keras.layers.CuDNNLSTM is deprecated. Please use tf.compat.v1.keras.layers.CuDNNLSTM instead.



In [6]:
model_conditions = 'Glu'
with open(os.path.join('..','..','data',model_conditions ,'training_data_'+model_conditions+'.txt')) as f:
    reader = csv.reader(f, delimiter="\t")
    d = list(reader)	

    

sequences = [di[0] for di in d]

for i in tqdm(range(0,len(sequences))) : 
    if (len(sequences[i]) > 110) :
        sequences[i] = sequences[i][-110:]
    if (len(sequences[i]) < 110) : 
        while (len(sequences[i]) < 110) :
            sequences[i] = 'N'+sequences[i]
            
            


# Function to embed sequences and another one to get reverse complements


seqdata_transformed = seq2feature(sequences)
print(seqdata_transformed.shape)



               



100%|██████████| 31349363/31349363 [00:19<00:00, 1606512.30it/s]


(31349363, 110, 4)


In [ ]:
with h5py.File(os.path.join('..','..','data',model_conditions ,'onehot_sequences_bool.h5'), 'w') as hf:
    hf.create_dataset("onehot_sequences_bool",  data=seqdata_transformed)
print(type(seqdata_transformed[0][0][0]))

#Reverse complement sequences are saved at a later stage below


## Now , Create The Data class label vectors and Store in the same h5py file
expressions = [di[1] for di in d]
expdata = np.asarray(expressions)
expdata = expdata.astype('float')  



with h5py.File(os.path.join('..','..','data',model_conditions ,'expression.h5'), 'w') as hf:
    hf.create_dataset("expression",  data=expdata)  


    


In [ ]:

################   
################  
#### Do the training test split below
################
################

if 0:
    ## Load the data matrix 
    with h5py.File(os.path.join('..','..','data',model_conditions ,'expression.h5'), 'r') as hf:
        expressions = hf['expression'][:]
    #expressions.shape

    ## Load the sequences 
    with h5py.File(join('..','..','data',model_conditions ,'onehot_sequences_bool.h5'), 'r') as hf:
        onehot_sequences = hf['onehot_sequences_bool'][:]
    #onehot_sequences.shape
    expdata = expressions
    seqdata_transformed = onehot_sequences


expressions = expdata
onehot_sequences = seqdata_transformed


## training, validation and test data split
## Shuffling 
if 0:
    randomize  =  np.random.permutation(len(onehot_sequences))
    onehot_sequences = onehot_sequences[randomize,:]
    expressions = expressions[randomize]


expressions = np.reshape(expressions , [-1,1])

fold_cv = 50
total_seqs = len(onehot_sequences)
_trX = onehot_sequences[int(total_seqs/fold_cv):]
_trY = expressions[int(total_seqs/fold_cv):]

_vaX = onehot_sequences[10000:int(total_seqs/fold_cv)]
_vaY = expressions[10000:int(total_seqs/fold_cv)]


_teX = onehot_sequences[0:10000]
_teY = expressions[0:10000]


_trX.shape , _trY.shape , _vaX.shape , _vaY.shape  






In [ ]:

with h5py.File(os.path.join('..','..','data',model_conditions ,'_trX.h5'), 'w') as hf:
    hf.create_dataset("_trX",  data=_trX)  
    
with h5py.File(os.path.join('..','..','data',model_conditions ,'_trY.h5'), 'w') as hf:
    hf.create_dataset("_trY",  data=_trY)  

with h5py.File(os.path.join('..','..','data',model_conditions ,'_vaX.h5'), 'w') as hf:
    hf.create_dataset("_vaX",  data=_vaX)  

with h5py.File(os.path.join('..','..','data',model_conditions ,'_vaY.h5'), 'w') as hf:
    hf.create_dataset("_vaY",  data=_vaY)  

with h5py.File(os.path.join('..','..','data',model_conditions ,'_teX.h5'), 'w') as hf:
    hf.create_dataset("_teX",  data=_teX)  

with h5py.File(os.path.join('..','..','data',model_conditions ,'_teY.h5'), 'w') as hf:
    hf.create_dataset("_teY",  data=_teY)  




### Generate standard scaler for positive and negative selection experiments using the extreme cases of the designed sequences (Done together since designed sequences are from the same file )

In [4]:
synthesized_seqs_filepath = os.path.join('..','..','data','synthesized_sequences_results',
                                    '20190325_NBT_MolEvol_seq_data_SCUra_YPD.splitByOrigID.meanEL.min100reads.txt')

def read_synthesized_sequences(filename) :

    with open(filename) as f:
        reader = csv.reader(f, delimiter="\t")
        d = list(reader)
    

    scura_exp = [di[3] for di in d]
    glu_exp = [di[4] for di in d]

    ### first row is nonsense
    scura_exp = scura_exp[1:]
    glu_exp = glu_exp[1:]


    return glu_exp,scura_exp

glu_exp,scura_exp = read_synthesized_sequences(synthesized_seqs_filepath)

def clean_exp(Y) :
    exp_NA = [(a=='NA') for a in Y]
    exp_NA = np.array(exp_NA)

    Y = np.array(Y)

    clean_exp = Y[~exp_NA]
    clean_exp = [float(a) for a in clean_exp ]
    return clean_exp
    
clean_glu_exp = np.array(clean_exp(glu_exp)).reshape(-1, 1)
clean_scura_exp = np.array(clean_exp(scura_exp)).reshape(-1, 1)



#save Glu scaler 
glu_scaler = sklearn.preprocessing.StandardScaler()
glu_scaler.fit(clean_glu_exp)
sklearn.externals.joblib.dump(glu_scaler, os.path.join('..','..','data','Glu','scaler.save' ) )

#save SC_Ura scaler 
scura_scaler = sklearn.preprocessing.StandardScaler()
scura_scaler.fit(clean_scura_exp)
sklearn.externals.joblib.dump(scura_scaler, os.path.join('..','..','data','SC_Ura','scaler.save' ) )

#scaler = sklearn.externals.joblib.load(scaler_filename) 


['../../data/SC_Ura/scaler.save']